In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Negamax Search

### Overview
According to [Knuth, D. E. and Moore, R. W. (1975)](Bibliography.ipynb#KM75), the **negamax search algorithm** is a simple algorithm for choosing an optimal move in a two-person zero-sum game with perfect information.

As the name implies, the negamax search algorithm utilizes a search tree. In this tree, each node represents a state of the game and the player who can make a move in that state. The edges of the tree represents moves that transition one state into another. Therefore, if a node representing state $\texttt{s} \in \texttt{States}$ and player $\texttt{p} \in \texttt{Players}$ has $\texttt{n} \in \mathbb{N} \cup \{0\}$ children, those child nodes represent the states $\texttt{ns}_1, ..., \texttt{ns}_\texttt{n}$ that can be reached when $\texttt{p}$ makes a legal move in $\texttt{s}$.

Negamax search assumes that both players in the game play optimally. In other words, each player is assumed to choose the best available legal move in any given game state. Because the $\texttt{heuristic}$ function returns negated scores for the opponent player (see [Introduction.ipynb](Introduction.ipynb#Heuristics)), the original player chooses the move with the highest score whereas the opponent player chooses the move with the lowest score. This is slightly different from the negamax search algorithm described by [Knuth, D. E. and Moore, R. W. (1975)](Bibliography.ipynb#KM75), where both players try to maximize the negated scores from their opponent. These two methods are functionally equivalent, but the former method allows scores to always represent the effectiveness of a move from the perspective of the player that started the search. This makes the scores more intuitive, therefore our implementation uses this method, despite carrying the drawback of making the implementation slightly more complex.

### Switching Turns
For negamax search, we require functions to switch turns. This involves both switching the current player and switching whether or not that player is the opponent. We call that first function $\texttt{switchPlayer}$, which has the following signature:

$$\texttt{switchPlayer: Players} \rightarrow \texttt{Players}$$

and is defined as follows:

$$\texttt{switchPlayer}(\texttt{Players}[1]) := \texttt{Players}[2]$$  
$$\texttt{switchPlayer}(\texttt{Players}[2]) := \texttt{Players}[1]$$

We call the second function $\texttt{switchOpponent}$, which has the following signature:

$$\texttt{switchOpponent:} \{-1, 1\} \rightarrow \{-1, 1\}$$

and is defined as follows:

$$\texttt{switchOpponent}(1) := -1$$  
$$\texttt{switchOpponent}(-1) := 1$$

### Paths
We define a *path* as a list $[(\texttt{s}_1, \texttt{m}_1, \texttt{p}_1, \texttt{o}_1), ..., (\texttt{s}_\texttt{n}, \texttt{m}_\texttt{n}, \texttt{p}_\texttt{n}, \texttt{o}_\texttt{n})]$ where each element is a tuple $\texttt{e} \in \texttt{States} \times \texttt{Moves} \times \texttt{Players} \times \{-1, 1\}$ representing information at a search node. Here, for all $\texttt{i} \in \{1, ..., \texttt{n} - 1\}$, the following statements hold:
- $\texttt{s}_{\texttt{i}+1} = \texttt{nextState}(\texttt{s}_\texttt{i}, \texttt{m}_\texttt{i}, \texttt{p}_\texttt{i})$
- $\texttt{m}_\texttt{i} \in \texttt{legalMoves}(\texttt{s}_\texttt{i}, \texttt{p}_\texttt{i})$
- $\texttt{p}_{\texttt{i}+1} = \texttt{switchPlayer}(\texttt{p}_\texttt{i})$
- $\texttt{o}_{\texttt{i}+1} = \texttt{switchOpponent}(\texttt{o}_\texttt{i})$

A path represents a path down the search tree. We define $\texttt{Paths}$ as the set of all valid paths.

### Finding the Best Value: Idea
We declare a function $\texttt{bestValue}$ with the following signature:

$$\texttt{bestValue: States} \times \texttt{Players} \times \{-1, 1\} \times \mathbb{N} \cup \{0\} \times \texttt{Paths} \rightarrow \mathbb{Z}$$

$\texttt{bestValue}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path})$ denotes the greatest value achievable from state $\texttt{s}$ given optimal play from both players, from the perspective of the player $\texttt{p}$ who can currently make a move. $\texttt{o} \in \{-1, 1\}$ is $\texttt{-1}$ if $\texttt{p}$ is the opponent player and $\texttt{1}$ otherwise. $\texttt{d} \in \mathbb{N} \cup \{0\}$ represents the current search depth, i.e. the number of half-moves to look into the future from the current position. $\texttt{path} \in \texttt{Paths}$ represents the path leading up to the current node.


### Finding the Next Values
As a helper function, we define the function $\texttt{nextValues}$ with the following signature:

$$\texttt{nextValues: States}\times \texttt{Players} \times \{-1, 1\} \times \mathbb{N} \times \texttt{Paths} \rightarrow 2^\mathbb{Z}$$

$\texttt{nextValues}$ returns the set of values from calling $\texttt{bestValue}$ on all states in the next layer in the search tree, i.e. on all states that can be reached when player $\texttt{p}$ makes a legal move in state $\texttt{s}$. This is the set that the player $\texttt{p}$ picks the highest or lowest value from, depending on whether or not they're the opponent. We define $\texttt{nextValues}$ as follows:

$
\begin{array}{l}
\texttt{nextValues}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}) := \{ \\
    \hspace{10mm} \texttt{bestValue}( \\
    \hspace{20mm} \texttt{nextState}(\texttt{s}, \texttt{m}, \texttt{p}), \\
    \hspace{20mm} \texttt{switchPlayer}(\texttt{p}), \\
    \hspace{20mm} \texttt{switchOpponent}(\texttt{o}), \\
    \hspace{20mm} \texttt{d} - 1, \\
    \hspace{20mm} \texttt{path} + [(\texttt{s}, \texttt{m}, \texttt{p}, \texttt{o})] \\
    \hspace{10mm} ) \mid \texttt{m} \in \texttt{legalMoves}(\texttt{s}, \texttt{p}) \\
\}
\end{array}
$

### Path Score
If the search depth $\texttt{d}$ reaches $0$ or the game has finished, we return the combined value from the $\texttt{heuristic}$ function for the move sequence leading up to the current of the search tree. In other words, this is the combined score for all moves in the path leading up to the current node. For this, we define the function $\texttt{pathScore}$ with the following signature:

$$\texttt{pathScore: Paths} \rightarrow \mathbb{Z}$$

We define $\texttt{pathScore}$ as follows:

$$ \texttt{pathScore}(\texttt{path}) := \sum_{i=1}^{\texttt{length}(\texttt{path})} \texttt{heuristic}(\texttt{s}_\texttt{i}, \texttt{m}_\texttt{i}, \texttt{p}_\texttt{i}, \texttt{o}_\texttt{i}) $$

where $\texttt{path}[\texttt{i}] = (\texttt{s}_\texttt{i}, \texttt{m}_\texttt{i}, \texttt{p}_\texttt{i}, \texttt{o}_\texttt{i})$ $\forall \texttt{i} \in \{1, ..., \texttt{length}(\texttt{path})\}$.

### Finding the Best Value: Definition
If the search depth is greater than $0$ and the game has not finished, $\texttt{bestValue}$ evaluates the moves at the next depth, maximizing or minimizing the score depending on whether or not $\texttt{p}$ is the opponent player, as depicted by the value $\texttt{o}$. Otherwise, it returns the path score as defined above to approximate how effective this sequence of moves is. With that, we can define $\texttt{bestValue}$ as follows:

$
\texttt{bestValue}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}) = 
\begin{cases}
\texttt{pathScore}(\texttt{path}) & \texttt{if finished}(\texttt{s}) \vee \texttt{d} = 0 \\
\max(\{\texttt{nextValues}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path})\}) & \texttt{if } \neg \texttt{finished}(\texttt{s}) \wedge \texttt{d} > 0 \wedge \texttt{o} = 1 \\
\min(\{\texttt{nextValues}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path})\}) & \texttt{if } \neg \texttt{finished}(\texttt{s}) \wedge \texttt{d} > 0 \wedge \texttt{o} = -1 \\
\end{cases} 
$

### Our Implementation
In our implementation, $\texttt{bestValue}$ is called `NegamaxSearch.get_best_move`. On top of the score of the optimal move, it also returns that optimal move as well as whether or not the endgame tablebase was used to find the move. It has the following arguments:
- `ai_turn`: Whether or not it's the turn of the player (AI) that started the search. This has the same functionality as the parameter $\texttt{o}$, where `ai_turn = True` is equivalent to $\texttt{o} = 1$ and `ai_turn = False` is equivalent to $\texttt{o} = -1$.
- `depth`: The remaining search depth. Identical to the parameter $\texttt{d}$.
- `last_eval_score`: The move score tied to the parent node. Our heuristic function is iterative, see [Evaluation.ipynb](Evaluation.ipynb#EvaluateMove). Therefore, the search needs to keep track of the score as it moves down the tree, rather than calculating the score at a leaf node. This uses the same idea as the function $\texttt{pathScore}$.

The parameters $\texttt{s}$ and $\texttt{p}$ are derived from `self.board` and `self.board.turn`, as this is a function of the `NegamaxSearch` class, which inherits from the `ChessAlgorithm` class and therefore has access to a `chess.Board` object containing the chess board state. The value $\texttt{path}$ is represented by the variable `best_move_path`. This only keeps track of the moves in the path, as the other elements are represented by the arguments and variables listed above and are updated throughout the search.

Optionally, this function uses **memoization**, sometimes called **transposition tables**, which is a type of caching. The idea is that, when a certain board state at a certain search depth is evaluated using the heuristic function, this result can be cached. If, later in the search, this board state is reached again at the same search depth, the result can simply be read from the cache, rather than having to evaluate it again.

Furthermore, this function implements **singular value extension**. If there's only one legal move to be made, the computation time of the current node is neglegible. Therefore, if this is the case, this node does not count towards the search depth, and the algorithm effectively searches one layer deeper into this branch.

### Dependencies

In [ ]:
import chess
import random

import import_ipynb
import Util
import Evaluation
from Globals import *
from ChessAlgorithm import ChessAlgorithm

### NegamaxSearch Class
A class that implements the negamax search algorithm as defined above, without memoization.

In [ ]:
class NegamaxSearch(ChessAlgorithm):
    pass

#### NegamaxSearch.get_best_move
Finds the best move to make based on the aforementioned negamax search algorithm, as well as the associated move score and whether or not the endgame tablebase was used.

__This function is implemented recursively.__

###### __<u>Arguments</u>__

``ai_turn (bool):``  
Whether or not the current turn is of the AI that started the search. If this is the case, the score should be maximized. Otherwise, the score should be minimized.   

``depth (int):``  
The remaining depth of the search, i.e. how many half-moves the search should still look into the future.  

``last_eval_score (int):``  
The score provided by the previous evaluation in the search. 

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The recommended best move to make.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

###### __<u>Side effects</u>__
If a cache dictionary for memoization is used, new values may be added to this dictionary.

In [ ]:
def get_best_move(
    self,
    ai_turn: bool,
    depth: int,
    last_eval_score: int
) -> (int, chess.Move, bool, list):

    if self.cache is not None:
        state = self.board.get_state_repr()
        if (depth, state) in self.cache:
            return self.cache[(depth, state)]

    color = self.board.turn
    original_color = color if ai_turn else not color
    result_score = self.board.get_search_result_if_finished(original_color,
            depth, last_eval_score)
    if result_score is not None:
        return result_score, None, False, []

    best_score = Globals.MAX_EVALUATION_SCORE * (-1 if ai_turn else 1)
    best_move = None
    best_move_used_endgame = False
    best_move_path = []
    use_singular_value_extension = (len(list(self.board.legal_moves)) == 1)
    
    for move in self.board.legal_moves:
        eval_score, used_endgame_anywhere = self.board.evaluate_move(
                self.use_heuristic, not ai_turn, last_eval_score,
                depth, move, self.endgame_tablebase)
        self.board.push(move)
        
        if self.board.is_game_over():
            score_after_move = eval_score
            new_move_path = []
        else:
            new_depth = depth if use_singular_value_extension else depth - 1
            score_after_move, _, used_endgame, new_move_path = self.get_best_move(
                    not ai_turn, new_depth, eval_score)
            used_endgame_anywhere = used_endgame_anywhere or used_endgame

        self.board.pop()

        if (ai_turn and score_after_move > best_score) \
                or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
            best_move_used_endgame = used_endgame_anywhere
            best_move_path = [move] + new_move_path

    if self.cache is not None:
        cache_key = (depth, self.board.get_state_repr())
        self.cache[cache_key] = best_score, best_move, best_move_used_endgame, best_move_path

    return best_score, best_move, best_move_used_endgame, best_move_path

NegamaxSearch.get_best_move = get_best_move
del get_best_move

#### NegamaxSearch.make_move
Finds the best possible move according to the negamax search algorithm and pushes it onto the move stack.

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The move that was made.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

In [ ]:
def make_move(self) -> (int, chess.Move, bool, list):
    score, move, used_endgame, move_path = self.get_best_move(
        ai_turn=True, 
        depth=self.search_depth,
        last_eval_score=0 # The starting board score doesn't matter,
                          # it's evaluated by score difference
    )
    self.board.push(move)

    return score, move, used_endgame, move_path

NegamaxSearch.make_move = make_move
del make_move

### NegamaxSearchMemo Class
A class that implements the negamax search algorithm as defined above, including memoization.

In [ ]:
class NegamaxSearchMemo(NegamaxSearch):
    pass

#### NegamaxSearchMemo.make_move
Finds the best possible move according to the negamax search algorithm and pushes it onto the move stack. The algorithm is optimized using memoization.

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The move that was made.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

In [ ]:
def make_move(self) -> (int, chess.Move, bool, list):
    self.cache = {}
    score, move, used_endgame, move_path = self.get_best_move(
        ai_turn=True, 
        depth=self.search_depth,
        last_eval_score=0 # The starting board score doesn't matter,
                          # it's evaluated by score difference
    )
    self.board.push(move)

    return score, move, used_endgame, move_path

NegamaxSearchMemo.make_move = make_move
del make_move